In [1]:
%run ../Python_files/util.py
%run ../Python_files/load_dicts.py

No dicts found; please check load_dicts...


In [2]:
import numpy as np
from numpy.linalg import inv

# load the original link counts data

import json

with open('../temp_files/link_day_minute_Apr_dict_journal_JSON_adjusted.json', 'r') as json_file:
    link_day_minute_Apr_dict_JSON_adjusted = json.load(json_file)

AM_flow_list = []
MD_flow_list = []
PM_flow_list = []
NT_flow_list = []
AM_flow_minute_list = []
MD_flow_minute_list = []
PM_flow_minute_list = []
NT_flow_minute_list = []
for link_idx in range(258):
    for day in range(31)[1:]: 
        key = 'link_' + str(link_idx) + '_' + str(day)
        AM_flow_list.append(link_day_minute_Apr_dict_JSON_adjusted[key] ['AM_flow'])
        MD_flow_list.append(link_day_minute_Apr_dict_JSON_adjusted[key] ['MD_flow'])
        PM_flow_list.append(link_day_minute_Apr_dict_JSON_adjusted[key] ['PM_flow'])
        NT_flow_list.append(link_day_minute_Apr_dict_JSON_adjusted[key] ['NT_flow'])
        for minute_idx in range(120):
            AM_flow_minute_list.append(link_day_minute_Apr_dict_JSON_adjusted[key] ['AM_flow_minute'][minute_idx])
            MD_flow_minute_list.append(link_day_minute_Apr_dict_JSON_adjusted[key] ['MD_flow_minute'][minute_idx])
            PM_flow_minute_list.append(link_day_minute_Apr_dict_JSON_adjusted[key] ['PM_flow_minute'][minute_idx])
            NT_flow_minute_list.append(link_day_minute_Apr_dict_JSON_adjusted[key] ['NT_flow_minute'][minute_idx])

x_AM_flow = np.matrix(AM_flow_list)
x_AM_flow = np.matrix.reshape(x_AM_flow, 258, 30)
x_AM_flow = np.nan_to_num(x_AM_flow)

x_MD_flow = np.matrix(MD_flow_list)
x_MD_flow = np.matrix.reshape(x_MD_flow, 258, 30)
x_MD_flow = np.nan_to_num(x_MD_flow)

x_PM_flow = np.matrix(PM_flow_list)
x_PM_flow = np.matrix.reshape(x_PM_flow, 258, 30)
x_PM_flow = np.nan_to_num(x_PM_flow)

x_NT_flow = np.matrix(NT_flow_list)
x_NT_flow = np.matrix.reshape(x_NT_flow, 258, 30)
x_NT_flow = np.nan_to_num(x_NT_flow)

x_AM_flow_minute = np.matrix(AM_flow_minute_list)
x_AM_flow_minute = np.matrix.reshape(x_AM_flow_minute, 258, 3600)
x_AM_flow_minute = np.nan_to_num(x_AM_flow_minute)

x_MD_flow_minute = np.matrix(MD_flow_minute_list)
x_MD_flow_minute = np.matrix.reshape(x_MD_flow_minute, 258, 3600)
x_MD_flow_minute = np.nan_to_num(x_MD_flow_minute)

x_PM_flow_minute = np.matrix(PM_flow_minute_list)
x_PM_flow_minute = np.matrix.reshape(x_PM_flow_minute, 258, 3600)
x_PM_flow_minute = np.nan_to_num(x_PM_flow_minute)

x_NT_flow_minute = np.matrix(NT_flow_minute_list)
x_NT_flow_minute = np.matrix.reshape(x_NT_flow_minute, 258, 3600)
x_NT_flow_minute = np.nan_to_num(x_NT_flow_minute)

zz1 = x_AM_flow_minute[:,3599] 
zz = x_AM_flow[:,29] 

In [3]:
zz1[0,0] + zz1[2,0] + zz1[4,0] -( zz1[1,0] + zz1[3,0] + zz1[5,0])

0.0

In [4]:
zz[0,0] + zz[2,0] + zz[4,0] -( zz[1,0] + zz[3,0] + zz[5,0])

-2.2737367544323206e-13

In [5]:
assert(zz1[0,0] + zz1[2,0] + zz1[4,0] - (zz1[1,0] + zz1[3,0] + zz1[5,0]) < 1e-5)
assert(zz1[6,0] - zz1[7,0] < 1e-5)
assert(zz1[1,0] + zz1[7,0] + zz1[8,0] + zz1[10,0] - (zz1[0,0] + zz1[6,0] + zz1[9,0] + zz1[11,0]) < 1e-5)
assert(zz1[12,0] + zz1[14,0] - (zz1[13,0] + zz1[15,0]) < 1e-5)

In [6]:
from collections import defaultdict
import csv

ini_node_dict = {}
ter_node_dict = {}

origi_dict = defaultdict(set)
desti_dict = defaultdict(set)

input_file = '../00_subnetwork_topology/MA_journal_topology.csv'

i = 0
with open(input_file, 'rb') as inp:
    for row in csv.reader(inp):
        if 'origin' not in row:
            ini_node_dict[i] = int(row[0])
            ter_node_dict[i] = int(row[1])
            origi_dict[int(row[0])].add(i)
            desti_dict[int(row[1])].add(i)
            i += 1

In [7]:
# for the journal network
def flow_conservation_adjustment_journal(y_0):
    L = len(y_0)  # dimension of flow vector x
    assert(L == 258)

    # y_0 = x[:,1]  # initial flow vector

    model = Model("Flow_conservation_adjustment_journal")

    y = []
    for l in range(L):
        y.append(model.addVar(name='y_' + str(l)))

    model.update() 

    # Set objective: ||y-y_0||^2
    obj = 0
    for l in range(L):
        obj += (y[l] - y_0[l]) * (y[l] - y_0[l])
    model.setObjective(obj)

    # Add nonnegative constraint: y >= 0
    for l in range(L):
        model.addConstr(y[l] >= 0)
    # Add flow conservation constraints
    for node in range(75)[1:]:
        model.addConstr(sum([y[link] for link in origi_dict[node]]) == \
                        sum([y[link] for link in desti_dict[node]])) 

    model.update() 

    model.setParam('OutputFlag', False)
    model.optimize()

    y = []
    for v in model.getVars():
        # print('%s %g' % (v.varName, v.x))
        y.append(v.x)
    # print('Obj: %g' % obj.getValue())
    return y

In [8]:
origi_dict[4], desti_dict[4]

({12, 14}, {13, 15})